In [2]:
import Pkg
Pkg.activate("./env")

  Activating project at `~/Studium/SCJ-PredatorPrey/env`


In [4]:
Pkg.add("Agents")
Pkg.add("AgentsExampleZoo")
Pkg.add("CairoMakie")
Pkg.add("GLMakie")

   Resolving package versions...
  No Changes to `~/Studium/SCJ-PredatorPrey/env/Project.toml`
  No Changes to `~/Studium/SCJ-PredatorPrey/env/Manifest.toml`
   Resolving package versions...
   Installed AgentsExampleZoo ─ v0.1.2
    Updating `~/Studium/SCJ-PredatorPrey/env/Project.toml`
  [88acaeb2] + AgentsExampleZoo v0.1.2
    Updating `~/Studium/SCJ-PredatorPrey/env/Manifest.toml`
  [88acaeb2] + AgentsExampleZoo v0.1.2
Precompiling project...
  ✓ AgentsExampleZoo
  1 dependency successfully precompiled in 4 seconds. 288 already precompiled.
   Resolving package versions...
  No Changes to `~/Studium/SCJ-PredatorPrey/env/Project.toml`
  No Changes to `~/Studium/SCJ-PredatorPrey/env/Manifest.toml`
   Resolving package versions...
  No Changes to `~/Studium/SCJ-PredatorPrey/env/Project.toml`
  No Changes to `~/Studium/SCJ-PredatorPrey/env/Manifest.toml`


In [13]:
# To view our starting population, we can build an overview plot using [`abmplot`](@ref).
# We define the plotting details for the wolves and sheep:
offset(a) = a isa Sheep ? (-0.1, -0.1*rand()) : (+0.1, +0.1*rand())
ashape(a) = a isa Sheep ? :circle : :utriangle
acolor(a) = a isa Sheep ? RGBAf(1.0, 1.0, 1.0, 0.8) : RGBAf(0.2, 0.2, 0.3, 0.8)

# and instruct [`abmplot`](@ref) how to plot grass as a heatmap:
grasscolor(model) = model.countdown ./ model.regrowth_time
# and finally define a colormap for the grass:
heatkwargs = (colormap = [:brown, :green], colorrange = (0, 1))

# and put everything together and give it to [`abmplot`](@ref)
plotkwargs = (;
    agent_color = acolor,
    agent_size = 25,
    agent_marker = ashape,
    offset,
    agentsplotkwargs = (strokewidth = 1.0, strokecolor = :black),
    heatarray = grasscolor,
    heatkwargs = heatkwargs,
)

(agent_color = acolor, agent_size = 25, agent_marker = ashape, offset = offset, agentsplotkwargs = (strokewidth = 1.0, strokecolor = :black), heatarray = grasscolor, heatkwargs = (colormap = [:brown, :green], colorrange = (0, 1)))

In [28]:
include("./predator_prey.jl")
Pkg.status(["Agents","GLMakie"]; mode = Pkg.Types.PKGMODE_MANIFEST, io=stdout)
using GLMakie
GLMakie.activate!()
stable_params = (;
    n_sheep = 140,
    n_wolves = 3,
    dims = (30, 30),
    Δenergy_sheep = 5,
    sheep_reproduce = 0.31,
    sheep_perception = 0,
    wolf_reproduce = 0.06,
    Δenergy_wolf = 30,
    wolf_perception = 0,
    seed = 71758,
)
params = Dict(
    :regrowth_time => 0:1:100,
    :Δenergy_sheep => 0:1:50,
    :sheep_reproduce => 0:0.01:1,
    :sheep_perception => 0:1:8,
    :Δenergy_wolf => 0:1:50,
    :wolf_reproduce => 0:0.01:1,
    :wolf_perception => 0:1:8,
)
sheep(a) = a isa Sheep
wolf(a) = a isa Wolf
count_grass(model) = count(model.fully_grown)
adata = [(sheep, count), (wolf, count)]
mdata = [count_grass]
model = initialize_model(;stable_params...)
fig, abmobs = abmexploration(model;params, plotkwargs...,  adata, alabels = ["Sheep", "Wolf"], mdata, mlabels = ["Grass"])
#fig, ax, abmobs = abmplot(model; add_controls=true, plotkwargs...)
fig

In [1]:
include("./predator_prey.jl")
CairoMakie.activate!() # hide
sheepwolfgrass = initialize_model(;stable_params...)

abmvideo(
    "sheepwolf.mp4",
    sheepwolfgrass;
    frames = 2000,
    framerate = 8,
    title = "Sheep Wolf Grass",
    plotkwargs...,
)

In [ ]:

sheepwolfgrass = initialize_model()

fig, ax, abmobs = abmplot(sheepwolfgrass; plotkwargs...)
fig

# Now, lets run the simulation and collect some data. Define datacollection:
sheep(a) = a isa Sheep
wolf(a) = a isa Wolf
count_grass(model) = count(model.fully_grown)
# Run simulation:
sheepwolfgrass = initialize_model()
steps = 1000
adata = [(sheep, count), (wolf, count)]
mdata = [count_grass]
adf, mdf = run!(sheepwolfgrass, steps; adata, mdata)

# The following plot shows the population dynamics over time.
# Initially, wolves become extinct because they consume the sheep too quickly.
# The few remaining sheep reproduce and gradually reach an
# equilibrium that can be supported by the amount of available grass.
function plot_population_timeseries(adf, mdf)
    figure = Figure(size = (600, 400))
    ax = figure[1, 1] = Axis(figure; xlabel = "Step", ylabel = "Population")
    sheepl = lines!(ax, adf.time, adf.count_sheep, color = :cornsilk4)
    wolfl = lines!(ax, adf.time, adf.count_wolf, color = RGBAf(0.2, 0.2, 0.3))
    grassl = lines!(ax, mdf.time, mdf.count_grass, color = :green)
    figure[1, 2] = Legend(figure, [sheepl, wolfl, grassl], ["Sheep", "Wolves", "Grass"])
    figure
end

plot_population_timeseries(adf, mdf)
